In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

load sample data

In [9]:
m = yaml.safe_load(open('../tests/data/fusion/C3L-00677.yaml'))
m

{'id': 'C3L-00677',
 'rna-seq': {'tumor': {'R1': {'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz'},
   'R2': {'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz'}}},
 'wxs': {'normal': {'barcode': 'CPT0071960002',
   'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'},
  'tumor': {'barcode': 'CPT0071100006',
   'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam'}}}

In [7]:
m['rna-seq']['tumor']['R1']['filepath'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz'
m['rna-seq']['tumor']['R2']['filepath'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz'
m['wxs']['tumor']['filepath'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam'
m['wxs']['normal']['filepath'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'




make test input yaml

In [10]:
template = yaml.safe_load(open('/home/estorrs/pecgs-pipeline/submodules/pecgs-fusion/cwl/template.fusion.yaml'))
template

{'bwts': {'class': 'Directory', 'path': 'a/file/path'},
 'cpu': 'a_int',
 'filter_database': {'class': 'Directory', 'path': 'a/file/path'},
 'fq_1': {'class': 'File', 'path': 'a/file/path'},
 'fq_2': {'class': 'File', 'path': 'a/file/path'},
 'fusion_annotator_dir': {'class': 'Directory', 'path': 'a/file/path'},
 'genome_db': {'class': 'Directory', 'path': 'a/file/path'},
 'genome_lib_dir': {'class': 'Directory', 'path': 'a/file/path'},
 'integrate_annotations': {'class': 'File', 'path': 'a/file/path'},
 'integrate_executable': {'class': 'File', 'path': 'a/file/path'},
 'integrate_fasta': {'class': 'File', 'path': 'a/file/path'},
 'sample': 'a_string'}

In [ ]:
sample = m['id']
fq_1 = m['rna-seq']['tumor']['R1']['filepath']
fq_2 = m['rna-seq']['tumor']['R2']['filepath']
cpu = 16
genome_lib_dir = '/data/pecgs/fusion/STAR-Fusion_dependencies/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/'
genome_db = '/data/pecgs/fusion/ericscript_dependencies/ericscript_db_homosapiens_ensembl84/'
# ericscript = '/data/pecgs/fusion/ericscript-0.5.5/ericscript.pl'
bwts = '/data/pecgs/fusion/Integrate_dependencies/bwts/'
integrate_fasta = '/data/pecgs/fusion/Integrate_dependencies/STAR/hg38.fa'
integrate_annotations = '/data/pecgs/fusion/Integrate_dependencies/annot.ensembl.GRCh38.txt'
combine_call_script = '/data/pecgs/fusion/Fusion_hg38_scripts/combine_call.pl'
filter_script = '/data/pecgs/fusion/Fusion_hg38_scripts/filter.pl'


In [11]:
template['bwts']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/Integrate_dependencies/bwts'
template['integrate_fasta']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/Integrate_dependencies/STAR/hg38.fa'
template['integrate_annotations']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/Integrate_dependencies/annot.ensembl.GRCh38.txt'
template['integrate_executable']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/INTEGRATE_0_2_6/INTEGRATE-build/bin/Integrate'
template['fusion_annotator_dir']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/FusionAnnotator'
template['filter_database']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/FilterDatabase'
template['genome_db']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/ericscript_dependencies/ericscript_db_homosapiens_ensembl84'
template['genome_lib_dir']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/STAR-Fusion_dependencies/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir'

template['sample'] = m['id']
template['cpu'] = 1
template['fq_1']['path'] = m['rna-seq']['tumor']['R1']['filepath']
template['fq_2']['path'] = m['rna-seq']['tumor']['R2']['filepath']


In [12]:
template

{'bwts': {'class': 'Directory',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/Integrate_dependencies/bwts'},
 'cpu': 1,
 'filter_database': {'class': 'Directory',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/FilterDatabase'},
 'fq_1': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz'},
 'fq_2': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz'},
 'fusion_annotator_dir': {'class': 'Directory',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/FusionAnnotator'},
 'genome_db': {'class': 'Directory',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/fusion/ericscript_dependencies/ericscript_db_h

In [13]:
yaml.safe_dump(template, open('../tests/data/fusion/C3L-00677_inputs.yaml', 'w'))

make cromwell compute1 config

In [14]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/fusion/C3L-00677'
output_fp = '../tests/data/fusion/C3L-00677.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

make execution script

In [15]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/fusion'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'C3L-00677.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'submodules', 'pecgs-fusion', 'cwl', 'fusion.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'C3L-00677_inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab/Active', '/storage1/fs1/m.wyczalkowski/Active']

In [16]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [17]:
start_commands, run_command = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes)
filepath = os.path.join('../tests/data/fusion/start_server.sh')
bsub.write_command_file(start_commands, filepath)
filepath = os.path.join('../tests/data/fusion/run_job.sh')
bsub.write_command_file([run_command], filepath)

In [18]:
for c in start_commands: print(c)

source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/fusion:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/fusion /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/fusion/C3L-00677:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/fusion/C3L-00677 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active"
bgadd -L 10 /estorrs/test_cromwell
bsub -q dinglab-interactive -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -Is '/bin/bash'


In [19]:
print(run_command)

/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/fusion/C3L-00677.cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar run -t cwl -i /storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/fusion/C3L-00677_inputs.yaml /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/pecgs-fusion/cwl/fusion.cwl
